In [1]:
# test full training of maskcnn_polished_with_rcnn_k_bl
from os.path import join

import numpy as np

import torch

from thesis_v2 import dir_dict

from thesis_v2.data.prepared.yuanyuan_8k import get_data

from thesis_v2.training_extra.maskcnn_like.opt import get_maskcnn_v1_opt_config
from thesis_v2.training_extra.maskcnn_like.training import (train_one,
                                                            partial)

from thesis_v2.models.maskcnn_polished_with_rcnn_k_bl.builder import (
    gen_maskcnn_polished_with_rcnn_k_bl,
    load_modules
)



In [2]:
def train_one_maskcnn_polished_with_rcnn_k_bl(
    split_seed,
    model_seed,
    act_fn,
    loss_type,
    input_size,
    out_channel,
    num_layer,
    kernel_size_l1,
    pooling_ksize,
    scale, scale_name,
    smoothness, smoothness_name,
    pooling_type,
    n_timesteps,
    multi_path_separate_bn,
):
    
    load_modules()
    datasets = get_data('a', 200, input_size, ('042318', '043018', '051018'), scale=0.5,
                        seed=split_seed)

    datasets = {
        'X_train': datasets[0].astype(np.float32),
        'y_train': datasets[1],
        'X_val': datasets[2].astype(np.float32),
        'y_val': datasets[3],
        'X_test': datasets[4].astype(np.float32),
        'y_test': datasets[5],
    }

    def gen_cnn_partial(input_size_cnn, n):
        return gen_maskcnn_polished_with_rcnn_k_bl(
                                    input_size=input_size_cnn,
                                    num_neuron=n,
                                    out_channel=out_channel,
                                    kernel_size_l1=kernel_size_l1,  # (try 5,9,13)
                                    kernel_size_l23=3,
                                    act_fn=act_fn,
                                    pooling_ksize=pooling_ksize,  # (try, 1,3,5,7)
                                    pooling_type=pooling_type,  # try (avg, max)  # looks that max works well here?
                                    num_layer=num_layer,
                                    n_timesteps=n_timesteps,
                                    factored_constraint=None,
                                    blstack_pool_ksize=1,
                                    blstack_pool_type=None,
                                    acc_mode='cummean',
                                    bn_after_fc=False,
                                    ff_1st_block=True,
            multi_path=True,
            multi_path_separate_bn=multi_path_separate_bn,
            # tried le5, gave same results as `test_full_training_ff_1st_multipath_train`
            multi_path_hack = 'leD3',
                                    )

    opt_config_partial = partial(get_maskcnn_v1_opt_config,
                                 scale=scale,
                                 smoothness=smoothness,
                                 group=0.0,
                                 loss_type=loss_type,
                                 )
    
    result = train_one(
        arch_json_partial=gen_cnn_partial,
        opt_config_partial=opt_config_partial,
        datasets=datasets,
        key=f'debug/test_full_training_maskcnn_polished_with_rcnn_k_bl/ff_1st_block_multipath_train/sep_bn{multi_path_separate_bn}_leD3/{model_seed}',
        show_every=100,
        max_epoch=40000,
        model_seed=model_seed,
        return_model=False,
        extra_params={
            # reduce on batch axis
            'eval_fn': {
                'yhat_reduce_axis': 1,
            }
        },
        print_model=True
    )
    
    return result['stats_best']['stats']['test']['corr_mean']

In [3]:
maskcnn_param_template = {
    'out_channel': 16,
    'num_layer': 3,
    'kernel_size_l1': 9,
    'pooling_ksize': 3,
    'pooling_type': 'avg',
    'model_seed': 0,
    'split_seed': 'legacy',
}

maskcnn_param_regular = {
    **maskcnn_param_template,
    **{
        'act_fn': 'relu',
        'loss_type': 'mse',
        'smoothness': 0.000005,
        'smoothness_name': '0.000005',
        'scale': 0.01,
        'scale_name': '0.01',
        'input_size': 50,
        'n_timesteps': 4,
    }
}

# print(train_one_maskcnn_polished_with_rcnn_k_bl(**maskcnn_param_regular, multi_path_separate_bn=False))

# 27629 = 1*2 + 16*1*9*9 + 16*2 + 2*(16*16*3*3+16*16*3*3+4*16*2) + 79*(14*14 + 16 + 1) 

# results here are exactly the same as those in `scripts/debug/rcnn_basic_kriegeskorte/test_full_training_ff_1st.ipynb`

In [4]:
print(train_one_maskcnn_polished_with_rcnn_k_bl(**maskcnn_param_regular, multi_path_separate_bn=True))

{'final_act', 'pooling', 'bn_output', 'fc'}
['conv0', 'bl_stack.layer_list.0.b_conv', 'bl_stack.layer_list.1.b_conv']
num_param 29613
JSONNet(
  (moduledict): ModuleDict(
    (accumulator): RecurrentAccumulator()
    (act0): ReLU()
    (bl_stack): BLConvLayerStack(
      (layer_list): ModuleList(
        (0): BLConvLayer(
          (b_conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (l_conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (1): BLConvLayer(
          (b_conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (l_conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
      )
      (bn_layer_list): ModuleList(
        (0): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (

0-0, train loss 0.1443062275648117
train loss 0.1443062275648117
val metric {'loss': 0.14617048501968383, 'loss_no_reg': 0.14376786351203918, 'corr': [0.0, 0.0, 0.0, -0.044446496495231275, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.07124348309139919, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.051931935038466284, -0.10188888848884817, -0.05308089244181002, 0.0, 0.0, 0.0, 0.0, 0.021506735929772716, 0.06134196192755239, 0.0, 0.0, 0.0, 0.03049137507294005, 0.0, 0.0, -0.019125879954220666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.040459644679946075, 0.0, 0.0, 0.03976508194558501, 0.0, 0.0, -0.029561403917653578, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00946054054482032, 0.0, 0.0, 0.0, 0.0, -0.2187269482809438, 0.0, 0.11438818706828995, 0.0, 0.0, 0.0327169933385157, 0.000989843008705124, 0.0, 0.0, 0.0, 0.0, 0.1964743168695262, -0.02605207757376831, 0.0, 0.0, 0.0, 0.0, 0.15677097469843732, 0.0], 'corr_mean': 0.0014082560825163287, 'corr_mean_neg': -0.0014082560825163287, 'corr2_mean': 0.002006392695203114, 'corr2_

========starting epoch 300==========
300-0, train loss 0.11839597672224045
train loss 0.11839597672224045
val metric {'loss': 0.12002036869525909, 'loss_no_reg': 0.11479201167821884, 'corr': [0.24082215785922384, 0.48966062074981254, 0.5305569594204742, 0.7430141439927129, 0.4513862823682045, 0.5053026647056167, 0.3965475416029817, 0.5633162563511018, 0.652611235242958, 0.67237605935226, 0.6470326528868882, 0.4689408043242441, 0.5648097971100452, 0.6296432102046297, 0.5381462335222329, 0.5862710228403848, 0.6761375956522195, 0.5929916414399953, 0.5486559882287618, 0.6573427769807861, 0.4250955084369375, 0.4106458786842209, 0.2357417546757698, 0.10864133712669372, 0.7500073051754546, 0.5118596019817803, 0.47228996946380314, 0.3209710655569896, 0.6370846496917983, 0.5270218381730662, 0.29426185167252783, 0.5898401814162938, 0.4120267831812813, 0.53039733219575, 0.6170512440225806, 0.4813980696526585, 0.542561826504401, 0.5652190787368997, 0.4192841352622621, 0.39320361617421146, 0.327387

========starting epoch 600==========
600-0, train loss 0.10895277559757233
train loss 0.10895277559757233
val metric {'loss': 0.11431891620159149, 'loss_no_reg': 0.10942494869232178, 'corr': [0.27228939069779406, 0.4992913256461874, 0.6054113782679114, 0.7652901790831874, 0.5277660617684756, 0.521106537671029, 0.4797155453918558, 0.6105890145412671, 0.6836904636025046, 0.7167714066834833, 0.6994821697838096, 0.5248370205843235, 0.6047816596635615, 0.645214845297416, 0.64260527676675, 0.6035740316606163, 0.7045542030889714, 0.6102713606076242, 0.5929858164906712, 0.6943489322317038, 0.45969501251553385, 0.4442287490564263, 0.3030289217782229, 0.09960855733252666, 0.7690135263161822, 0.5945035015839286, 0.5793031791934784, 0.3739526351875549, 0.6669854800411402, 0.561000306793553, 0.34797805117753355, 0.581897058535904, 0.5062879098416586, 0.5698070994548065, 0.63548770485484, 0.5326573443344016, 0.6139121312353846, 0.6199804268960004, 0.46842250130011803, 0.40663696490373863, 0.41261984

========starting epoch 900==========
900-0, train loss 0.10974754393100739
train loss 0.10974754393100739
val metric {'loss': 0.11239622831344605, 'loss_no_reg': 0.10767494142055511, 'corr': [0.2655422074996363, 0.4955410304856157, 0.6057569160023717, 0.7560376066460871, 0.522588049256052, 0.5130031164373081, 0.4914938698132155, 0.6272391522551696, 0.6889767684854828, 0.7163394516672774, 0.6981648170887198, 0.5360134074922032, 0.6113176013479064, 0.6564247024019172, 0.6511892348386988, 0.6101108448957764, 0.7144588440915043, 0.6092329867677234, 0.5926656339435228, 0.6927085690808699, 0.4698604815663913, 0.4455424615027883, 0.341764201150615, 0.09894966502981133, 0.7694971276156799, 0.6072610871726454, 0.5817404144725686, 0.37801755730320835, 0.6706748141475091, 0.5655814441476866, 0.3492076874343629, 0.579819542892287, 0.5193074842758034, 0.5681424978701703, 0.6393708422594119, 0.529895044634791, 0.6339566715614905, 0.64218653785742, 0.4798252257782023, 0.4161167762338191, 0.4429265296

...

========starting epoch 400==========
400-0, train loss 0.10388478636741638
train loss 0.10388478636741638
val metric {'loss': 0.11003468632698059, 'loss_no_reg': 0.10584288835525513, 'corr': [0.27073148990256973, 0.5122043534785324, 0.6277014726637536, 0.7611534158969546, 0.5416529164347335, 0.5225264740240265, 0.5130993348723338, 0.653101715335769, 0.6918333917844144, 0.7270433734566273, 0.7067392899413112, 0.5485855840512368, 0.621517954408714, 0.6532290450802468, 0.6595719822676974, 0.6184553956080826, 0.7256970472433741, 0.6241531340621749, 0.6139800749520536, 0.703441680647529, 0.4827981041919986, 0.4555274080012144, 0.3826541004100047, 0.11200898581565324, 0.7781038365542678, 0.6419198260387156, 0.591944280010382, 0.3888106008529757, 0.6731362674798793, 0.5625832720753239, 0.38455305604869816, 0.5917872420594403, 0.5307737032569679, 0.604010843657705, 0.6461789515947612, 0.5319465662685541, 0.6405772309827519, 0.6657768316792478, 0.5056168283695552, 0.41340032001591664, 0.4510083

========starting epoch 700==========
700-0, train loss 0.1006242111325264
train loss 0.1006242111325264
val metric {'loss': 0.11033628582954406, 'loss_no_reg': 0.10621291399002075, 'corr': [0.27669785282994896, 0.515316167416465, 0.6280011482569099, 0.760301715381489, 0.5380715262092086, 0.5226239030903038, 0.5121496850359246, 0.6533041077006032, 0.6911159502810307, 0.7291403264625667, 0.7074618967214998, 0.5491909537605737, 0.6220845695451356, 0.6541235713437034, 0.6591911026049243, 0.6197890831677975, 0.7251495581226781, 0.6210121247630154, 0.6153379687283982, 0.7051360682171586, 0.4837125144407102, 0.4548778746771289, 0.38153448064896467, 0.11343559675413648, 0.7763204444623542, 0.6418022365291958, 0.5890181879961944, 0.3896941765628707, 0.6742080559351925, 0.5638922219131499, 0.38244310748193044, 0.5942356720896838, 0.5322070531786991, 0.603844541246205, 0.6451819193987738, 0.5297788806501778, 0.6454964727847025, 0.6616891647766258, 0.5071552510417204, 0.41088565612746386, 0.451830

========starting epoch 100==========
100-0, train loss 0.10506466031074524
train loss 0.10506466031074524
val metric {'loss': 0.10987830609083175, 'loss_no_reg': 0.10576222091913223, 'corr': [0.2698635465474989, 0.515920073183799, 0.6290976776990456, 0.7616288997986524, 0.539475841804244, 0.5228432614590733, 0.5114359550028558, 0.6529355095673357, 0.6930574003689476, 0.7286657719221937, 0.7062836507610873, 0.5494645852974652, 0.6219526720113822, 0.651676647862533, 0.6589303092111193, 0.6194858458947983, 0.7253502335826855, 0.6244195446804879, 0.6154344649147352, 0.7061164490076515, 0.4813720593845998, 0.4545932107624179, 0.38026802590520037, 0.11255444756700639, 0.7784724562092598, 0.640252191272703, 0.592224074001193, 0.39000160489668173, 0.6730820573245745, 0.5650439790855329, 0.38211459561225747, 0.5916016188997425, 0.5317634298729319, 0.602295608024707, 0.6472139467289495, 0.5331379898072163, 0.6448557245680588, 0.6637547966459669, 0.5059443564365498, 0.4091043696644172, 0.45424700

========starting epoch 400==========
400-0, train loss 0.10083360970020294
train loss 0.10083360970020294
val metric {'loss': 0.10982789397239685, 'loss_no_reg': 0.10569710284471512, 'corr': [0.2691447897496235, 0.5150371338945938, 0.6277020037229238, 0.7625209062474152, 0.5383681738975189, 0.5229631766890088, 0.5119247297611587, 0.6517714539301922, 0.6925335958091136, 0.7285504925352644, 0.7069646792358215, 0.5486142986647788, 0.6222296648023514, 0.6535084384141795, 0.6602807020275067, 0.619839586716428, 0.7243388473524348, 0.6246852235371418, 0.6145675344045531, 0.7053066811670196, 0.4813537204192289, 0.4550440426490058, 0.38118174728075394, 0.11210926442358878, 0.7779253827180643, 0.6407298978537728, 0.5931571305461923, 0.39064935368393483, 0.6744655961451175, 0.5645054695634548, 0.3831381850242369, 0.5922769285552081, 0.5340026121154149, 0.6025310044574859, 0.6470544031780933, 0.5327429098030816, 0.645348075763442, 0.6637867808930829, 0.506413464615801, 0.4116452769320589, 0.455151

========starting epoch 700==========
700-0, train loss 0.10096006840467453
train loss 0.10096006840467453
val metric {'loss': 0.10994871705770493, 'loss_no_reg': 0.10581935197114944, 'corr': [0.27030379557975315, 0.514730957367114, 0.6277889997228692, 0.7615756479883158, 0.5383207878500271, 0.522098564609013, 0.5114244198566471, 0.6530410229437509, 0.6928177011961494, 0.7287569247232412, 0.7063658256083635, 0.5486447756851307, 0.6224809375332628, 0.6524805759338619, 0.6596174308804184, 0.61899999539001, 0.7241715511494671, 0.623644809966228, 0.6151394102002175, 0.7048243541525618, 0.4815582957551896, 0.4556540730962322, 0.3820860662161506, 0.11283100183606139, 0.7777277497950901, 0.6395891331953514, 0.592054667821255, 0.392222890153165, 0.6731006260254693, 0.5645242698008182, 0.3835107932300969, 0.5904013753659805, 0.5311902240253621, 0.6036532740542457, 0.646473571743946, 0.5317325513855315, 0.6442728286785442, 0.6640959766983634, 0.506860018352387, 0.4103322437378467, 0.4507780035018